In [5]:
from IPython.display import display, HTML; display(HTML("<style>.container { width:100% !important; }</style>"))
import threading;import time;import pynq.lib.rgbled as rgbled;from pynq.overlays.base import BaseOverlay;import asyncio
base = BaseOverlay("base.bit")

Names, dining_sequence = ['Descartes','Aristotle','Socrates ','Thoreau  ','Th. Paine'], [0, 2, 4, 1, 3]
State, ForkNumbers = ['STARVING','EATING','NAPPING','DONE'], [1, 2, 2, 3, 3, 4, 4, 5, 5, 1]
Philosophers, threads, forklocks = [], [], []
led4, led5 = rgbled.RGBLED(4), rgbled.RGBLED(5)
eating_time, napping_time, askagain_time = 10, 20, 0.3
eating_blinkingRate, napping_blinkingRate, askagain_blinkingRate = 0.03, 1, 0.03
sz = len(Names)
threads_flag = True    # "cond" will keep threads live or kill them all
class forks:
    def __init__(self, fork1, fork2):
        self.fork1 = fork1
        self.fork2 = fork2
class Philosopher:
    def __init__(self, name, state, forks):
        self.name = name
        self.state = state
        self.forks = forks
def leds(n, onoff):
    if n < 4:    base.leds[n].write(onoff)
    elif n == 4: led4.write( 0x2 * onoff ) # toggle and keep it green pls
    else:        led5.write( 0x2 * onoff )  
def blink(t, d, n):
    for i in range(t):       #[t]: times to blink the LED
        if n < 4:    base.leds[n].toggle()
        elif n == 4: led4.write( 0x2 * (not led4.read())) # toggle and keep it green pls
        else:        led5.write( 0x2 * (not led5.read()))

        time.sleep(d)        #[d]: duration (in seconds) for the LED to be on/off    
    if n < 4:    base.leds[n].off()
    elif n == 4: led4.write(0x0)
    else:        led5.write(0x0)     
def blinkbydelay(totalDelay, blinkingRate, n):
    times2blink = int(totalDelay/blinkingRate)
    blink(times2blink,blinkingRate,n)   
def CanIHaveBothForks(_l, forklocks, num):    
    fork1 = Philosophers[num].forks.fork1[1];fork2 = Philosophers[num].forks.fork2[1]
    if forklocks[fork1-1].locked() == False and forklocks[fork2-1].locked() == False:        
        forklocks[fork1-1].acquire();forklocks[fork2-1].acquire()
        print("{} fork {} and {} are available, now locked".format(Philosophers[num].name, fork1,fork2))
        return True
    else:#print("{} forks NOT available".format(Philosophers[num].name))        
        return False
def worker_t(_l, forklocks, num):# Worker function to try and acquire resource, [_l]: threading lock (resource), [num]: index representing the LED and thread number.
    while threads_flag:        
        if Philosophers[num].state == State[0]:#STARVING
            #print("{} Can I ask? ".format(Philosophers[num].name))
            if _l.locked() == False:#Are you available for asking for both forks?
                _l.acquire()
                res = CanIHaveBothForks(_l, forklocks, num)
                if res == True:# Here we have both forks!
                    leds(num,1)
                    Philosophers[num].state = State[1]#EATING   #move on machine state
                    print("{} changed state to: {}".format(Philosophers[num].name, State[1]))                                        
                else:
                    time.sleep(0.5)#was 3
                _l.release()
            else:
                time.sleep(askagain_time)              
        elif Philosophers[num].state == State[1]:#EATING
            print("{} Eating".format(Philosophers[num].name))            
            blinkbydelay(eating_time, eating_blinkingRate, num)#time.sleep(eating_time)
            Philosophers[num].state = State[2]#NAPPING
            print("{} Done Eating, changed state to: {}\n".format(Philosophers[num].name, State[2]))#NAPPING           
            # Relinquish forks
            fork1 = Philosophers[num].forks.fork1[1];fork2 = Philosophers[num].forks.fork2[1] 
            forklocks[fork1-1].release();forklocks[fork2-1].release()
            leds(num,0)
        elif Philosophers[num].state == State[2]:#NAPPING
            print("{} Napping".format(Philosophers[num].name))
            blinkbydelay(napping_time, napping_blinkingRate, num)#time.sleep(napping_time)
            Philosophers[num].state = State[3]#DONE
            print("\n{} Done Napping, changed state to: {}".format(Philosophers[num].name, State[3]))#DONE  
            leds(num,1)
        elif Philosophers[num].state == State[3]:#DONE
            time.sleep(1)

        time.sleep(1)        
async def get_btns(_loop):
    btns = base.btns_gpio
    global threads_flag, leds, btn0_flag, btn1_flag, btn2_flag, btn3_flag, Philosophers, Names, State, leds, dining_sequence
    while threads_flag:
        await asyncio.sleep(0.1)
        if btns.read() != 0:
            if (btns[0].read()==1):
                await asyncio.sleep(0.3)
                print("button 0 pressed, Restart the Simulation")
                time.sleep(0.1)
                print("Simulation Re-started")
                enum_dining_sequence = enumerate(dining_sequence)# launch philosophers' thread in an optimized sequence
                for idx, e in enum_dining_sequence:
                    Philosophers[idx].state = State[0]
                    time.sleep(0.2)
                    leds(idx,0)
                btn0_flag = True
            elif (btns[1].read()==1):                
                await asyncio.sleep(0.3)
                print("button 1 pressed")        
                btn0_flag = True
            elif (btns[2].read()==1):
                await asyncio.sleep(0.3)
                print("button 2 pressed")              
                btn0_flag = True
            else:
                await asyncio.sleep(0.3)
                btn3_flag = True
                print("button 3 pressed, Exit program")
                threads_flag = False
                await asyncio.sleep(1)
                Names = ['Descartes','Aristotle','Socrates ','Thoreau  ','Th. Paine']
                for name in enumerate(Names):#Show a Status Summary 
                    print(Philosophers[name[0]].name,"\thas\tForks ", Philosophers[name[0]].forks.fork1[1]," and ", Philosophers[name[0]].forks.fork2[1],"\tState: ", Philosophers[name[0]].state)                    
                print("Forks lock status: {} {} {} {} {}".format(forklocks[0].locked(),forklocks[1].locked(),forklocks[2].locked(),forklocks[3].locked(),forklocks[4].locked()))

                _loop.stop()

print("Press buttons 0-3\n   0 - Restart simulation\n   1 - No use\n   2 - No use\n   3 - Exit program")

enum_ForkNumbers = enumerate(ForkNumbers)   
for name in enumerate(Names):   
    neededforks = forks(next(enum_ForkNumbers),next(enum_ForkNumbers))    
    p = Philosopher(name[1], State[0],neededforks)   
    Philosophers.append(p)

for name in enumerate(Names):#Show a Status Summary 
    print(Philosophers[name[0]].name,"\thas\tForks ", Philosophers[name[0]].forks.fork1[1]," and ", Philosophers[name[0]].forks.fork2[1],"\tState: ", Philosophers[name[0]].state)

askingforboth = threading.Lock()#atomic locking/release
for n in range(5):
    leds(n,0)
    singlelock = threading.Lock()
    forklocks.append(singlelock)
    
for i in range(5):#Philosophers will show up to the table to be seated, every 2 seconds
    t = threading.Thread(target=worker_t, args=(askingforboth, forklocks, i))
    threads.append(t)
    
enum_dining_sequence = enumerate(dining_sequence)# launch philosophers' thread in an optimized sequence
for idx, e in enum_dining_sequence:
    threads[e].start()

loop = asyncio.new_event_loop();  # Instance event_loop object
loop.create_task(get_btns(loop)); # take user input buttons    

loop.run_forever()
loop.close()
    
for t in threads:
    t.join()
    print('{} joined'.format(t.name))

for name in enumerate(Names):#Show a Status Summary 
    print(Philosophers[name[0]].name,"\thas\tForks ", Philosophers[name[0]].forks.fork1[1]," and ", Philosophers[name[0]].forks.fork2[1],"\tState: ", Philosophers[name[0]].state)
    
print("Forks lock status: {} {} {} {} {}".format(forklocks[0].locked(),forklocks[1].locked(),forklocks[2].locked(),forklocks[3].locked(),forklocks[4].locked()))

##input("\n\t\t...press any key to exit...")
for n in range(5):
    leds(n,0)
print("Done.")     

Press buttons 0-3
   0 - Restart simulation
   1 - No use
   2 - No use
   3 - Exit program
Descartes 	has	Forks  1  and  2 	State:  STARVING
Aristotle 	has	Forks  2  and  3 	State:  STARVING
Socrates  	has	Forks  3  and  4 	State:  STARVING
Thoreau   	has	Forks  4  and  5 	State:  STARVING
Th. Paine 	has	Forks  5  and  1 	State:  STARVING
Descartes fork 1 and 2 are available, now locked
Descartes changed state to: EATING
Socrates  fork 3 and 4 are available, now locked
Socrates  changed state to: EATING
Socrates  Eating
Descartes Eating
Socrates  Done Eating, changed state to: NAPPING

Descartes Done Eating, changed state to: NAPPING

Socrates  Napping
Descartes Napping
Th. Paine fork 5 and 1 are available, now locked
Th. Paine changed state to: EATING
Aristotle fork 2 and 3 are available, now locked
Aristotle changed state to: EATING
Aristotle EatingTh. Paine Eating

Th. Paine Done Eating, changed state to: NAPPING

Aristotle Done Eating, changed state to: NAPPING

Thoreau   fork 4 a